In [1]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#import matplotlib.patches as mpatches
import os
import json
import math
from datetime import datetime

#from patsy import dmatrices

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.tree import export_graphviz

#import graphviz
#from graphviz import Source
#from graphviz import render

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
f = open('/Users/laura/Desktop/Trimester_3/research-project/web_app/journeyplanner/static/journeyplanner/ordered_stops_main.json',) 
  
# returns JSON object as  
# a dictionary 
stop_data = json.load(f) 

In [3]:
#getting the main subroute (we will need to make provision for the subroutes)
subroutes = pd.read_csv("/Users/laura/Desktop/Trimester_3/research-project/data_analytics/routes_subroutes-04072020.csv", keep_default_na=True, sep=',\s+', delimiter=',', skipinitialspace=True)


In [4]:
days = {0: "Monday", 1: "Tuesday", 2: "Wednesday", 3:"Thursday", 4:"Friday", 5:"Saturday", 6:"Sunday"}
months = {1: "January", 2: "Febuary", 3: "March", 4:"April", 5:"May", 6:"June", 7:"July", 8:"August", 9:"September", 10:"October", 11:"November", 12:"December"}
times = {0:"0", 1:"1", 2:"2", 3:"3", 4:"4", 5:"5", 6:"6", 7:"7", 8:"8", 9:"9", 10:"10", 11:"11", 12:"12", 13:"13", 14:"14", 15:"15", 16:"16", 17:"17", 18:"18", 19:"19", 20:"20", 21:"21", 22:"22", 23:"23", 24:"24", 25:"25", 26:"26", 27:"27", 28:"28"}

In [5]:
def main_subroute(route, direction):
    #print(subroutes['LINEID'])
    selection = subroutes[subroutes['LINEID']==str(route)]
    #print("selection head", selection.head())
    route_direction = selection[selection.DIRECTION == direction]
    #print("route_direction", route_direction.head())
    subroute_list = list(route_direction.MAINROUTE.unique())
    #print("subroute list", subroute_list)
    main_subroute = subroute_list[0][2:-2]
    return main_subroute

In [6]:
def segments(dataframe, list_stops):
    nan = 0
    journies = 0
    #print(dataframe.head())
    #print(list(dataframe["STOPPOINTID"].unique()))
    dataframe_stops = list(dataframe["STOPPOINTID"].unique())
    #print(dataframe_stops)
    segments = []
    means = []
    maxs = []
    mins = []
    stds = []
    medians = []
    proportions = []
    total_journeytime = 0
    segment_dict = {}
    for i in range(0,len(list_stops)-1):
        if list_stops[i] in dataframe_stops and list_stops[i+1] in dataframe_stops:
            segment = (str(list_stops[i]) + "_" + str(list_stops[i+1]))
            segments.append(segment)
        #print(list_stops[i])
        #print(type(list_stops[i]))
            only_first = dataframe[dataframe["STOPPOINTID"]==list_stops[i]]
        #print(list(only_first["STOPPOINTID"].unique()))
        #print(only_first.head())
            only_last = dataframe[dataframe["STOPPOINTID"]==list_stops[i+1]]
            for_merging = only_last[["ACTUALTIME_ARR", "UNIQUE_TRIP"]]
            for_merging.columns = ["ACTUALTIME_ARR_LAST", "UNIQUE_TRIP"]
        #print(for_merging.head())
            result = pd.merge(only_first,
                 for_merging[["ACTUALTIME_ARR_LAST", "UNIQUE_TRIP"]],
                 on='UNIQUE_TRIP', 
                 how='left')
            
            reduced_result = result.dropna()
                         
            reduced_result['ACTUALTIME_ARR_LAST'] = reduced_result['ACTUALTIME_ARR_LAST'].astype('int64')

            reduced_result['JOURNEYTIME'] = reduced_result["ACTUALTIME_ARR_LAST"] - reduced_result["ACTUALTIME_DEP"]
            
            
            #total_journeytime += reduced_result['JOURNEYTIME'].mean()
            means.append(reduced_result['JOURNEYTIME'].mean())
            maxs.append(reduced_result['JOURNEYTIME'].max())
            mins.append(reduced_result['JOURNEYTIME'].min())
            stds.append(reduced_result['JOURNEYTIME'].std(ddof=0))
            
            medians.append(reduced_result['JOURNEYTIME'].median())
    
    for i in means:
        if math.isnan(i):
            pass
        else:
            total_journeytime += i
            
            #print("Total journey time: ",total_journeytime)
    for i in means:
        if total_journeytime > 0:
            proportions.append(i/total_journeytime)
        
    means_dict = dict(zip(segments, means))
    max_dict = dict(zip(segments, maxs))
    min_dict = dict(zip(segments, mins))
    std_dict = dict(zip(segments, stds))
    median_dict = dict(zip(segments, medians))
    proportions_dict = dict(zip(segments, proportions))
    if nan > 0:
        print("NaN total:", nan)
        
    if journies > 0:
        print("Journey Error")
    
    segment_dict['means'] = means_dict
    segment_dict['max'] = max_dict
    segment_dict['min'] = min_dict
    segment_dict['std'] = std_dict
    segment_dict['medians'] = median_dict
    segment_dict['proportions'] = proportions_dict
                           
    return segment_dict

In [11]:
def function(df, list_stops):    
    dictionary = {}
    count=0
    dictionary = {}
    for day in days:
        day_of_week = days[day]
        temp1 = df[df['DAYOFWEEK']==day]
        for month in months:
            current_month = months[month]
            temp2 = temp1[temp1['MONTH']==month]
            for time in times:
                current_time = times[time]
                dictionary[day_of_week + "_" + current_month + "_" + current_time] = {}
                temp3 = temp2[temp2["TIME_GROUP"]==int(time)]
                if temp3.empty:
                    pass
                    #print('DataFrame is empty!')
                else:
                    #print("dataframe going into segments function: ", temp3.head())
                    result = segments(temp3, list_stops)
                    dictionary[day_of_week + "_" + current_month + "_" + current_time]= result
                    count+=1
                    if count%100 == 0:
                        print(count)
    return dictionary

In [7]:
files = os.listdir("/Users/laura/Desktop/Master_Route_Direction_3/")
files
print(len(files))

77


In [8]:
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(NpEncoder, self).default(obj)


In [9]:
def driver(file, route, direction):
    df = pd.read_csv(file, keep_default_na=True, sep=',\s+', delimiter=',', skipinitialspace=True)
    months = list(df["MONTH"].unique())
    days = list(df["DAYOFWEEK"].unique())
    times = list(df["TIME_GROUP"].unique())
    
    main_sub = main_subroute(route, direction)
    main_subroute_df = df[df['ROUTEID']==main_sub]
    
    main_route_json = json_data[main_sub]
    main_route_json_stops = main_route_json['stops']

    final_result = function(main_subroute_df, main_route_json_stops)
    return final_result

In [ ]:
for file in files:
    if str(file)[-4:] == ".csv":
        route_split = file.split("_", 1)
        #print(route_split)
        direction = int(file[-5:-4])
        route = route_split[0]
        json_data = stop_data[str(route)]
        print(route, direction, " in progress.")
        filepath = "/Users/laura/Desktop/Master_Route_Direction_4/" + str(file)
        result = driver(filepath, route, direction)
        
        filename = "/Users/laura/Desktop/proportions_test/" + str(route)+ '_' + str(direction) + "_" + "proportions.json"
        with open(filename, 'w') as outfile:
            json.dump(result, outfile, cls=NpEncoder)